In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Mesothelioma/GSE67487'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genomic  characterization of mesothelioma patients by CGH arrays."
!Series_summary	"Genomic  characterization of mesothelioma patients by CGH arrays."
!Series_summary	"Genomic  characterization of mesothelioma patients by CGH arrays."
!Series_overall_design	"The goal of this study was to determine the main genomic aberrations related to mesothelioma in a French cohorte."
Sample Characteristics Dictionary:
{0: ['sample type: mesothelioma'], 1: ['gender: male', 'gender: female'], 2: ['age (years): 67', 'age (years): 17', 'age (years): 75', 'age (years): 47', 'age (years): 61', 'age (years): 52', 'age (years): 48', 'age (years): 57', 'age (years): 69', 'age (years): 63', 'age (years): 31', 'age (years): 73', 'age (years): 76', 'age (years): 60', 'age (years): 43', 'age (years): 72', 'age (years): 49', 'age (years): 65', 'age (years): 68', 'age (years): 70', 'age (years): 42', 'age (years): 66'], 3: ['histological properties: Epithelial', 'histologica

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Data Availability and Type Determination
is_gene_available = True  # Assume gene expression data is available based on dataset specifications.

# Determine the availability of the 'Mesothelioma' trait
trait_row = None  # since 'sample type: mesothelioma' does not present variability

# Determine the availability of 'age' trait
age_values = set(['age (years): 67', 'age (years): 17', 'age (years): 75', 'age (years): 47', 'age (years): 61', 'age (years): 52', 'age (years): 48', 'age (years): 57', 'age (years): 69', 'age (years): 63', 'age (years): 31', 'age (years): 73', 'age (years): 76', 'age (years): 60', 'age (years): 43', 'age (years): 72', 'age (years): 49', 'age (years): 65', 'age (years): 68', 'age (years): 70', 'age (years): 42', 'age (years): 66'])
age_row = 2 if len(age_values) > 1 else None

# Determine the availability of 'gender'
gender_values = set(['gender: male', 'gender: female'])
gender_row = 1 if len(gender_values) > 1 else None 

# Data Type Conversion Functions
def convert_trait(value):
    return 1 if 'mesothelioma' in value.lower() else None

def convert_age(value):
    try:
        return float(value.split(':')[1].strip())
    except ValueError:
        return None

def convert_gender(value):
    gender = value.split(':')[1].strip().lower()
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    return None

# Save Metadata
save_cohort_info('GSE67487', './preprocessed/Mesothelioma/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Mesothelioma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Mesothelioma/trait_data/GSE67487.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
else:
    print("Clinical data extraction skipped as trait_row is None.")


Clinical data extraction skipped as trait_row is None.
